In [1]:
# make_seaice_climatology_file
#
# Makes the long-term mean
#
# Steven Cavallo
# December 2024

In [2]:
import numpy as np
import datetime as dt
import matplotlib as mpl
import matplotlib.pyplot as plt
import os, datetime, pylab
import pylab as P
from scipy.signal import butter, lfilter, filtfilt

from mstats import *
import utilities_modules as um

In [3]:
# Important!  fpath below just needs to be set to a daily sea ice file that ends with a complete years' worth of data that is past the desired climatology.  For example, to a compute a climatology through 2010, I just need a file with data through 2010, but the file must contain data for every day of the year. 
fpath = '/Users/scavallo/Documents/data/seaice_loss/paper_data_directory/N_seaice_extent_daily_v3.0_1979_2023.csv'
climo_path = '/Users/scavallo/Documents/data/seaice_loss/paper_data_directory/NH_seaice_dextent_int_longtermmean_errorbars_1981_2010.dat'
plot_variable = 'dextent_int' # 'extent','dextent', dextent_int, or dextent_diff
climo_years = [1981,2010]
filter_climo = 'True'
compute_errorbars = 'False'
errorbar_percentiles = [0,100]

In [4]:
if 1 == 1:
    file = open(fpath, 'r')
    data = np.loadtxt(file,skiprows=2, delimiter=',',usecols=(0,1,2,3))
    years = data[:,0].astype(int) #e.g., 1981
    months = data[:,1].astype(int) #e.g., 01
    days=data[:,2].astype(int) #e.g., 01
    extents = data[:,3] #e.g., 11.345 [10^6km^2]    
    
    #convert individual yr,month,day to time values
    nVals = len(extents)
    print(nVals)
    timeList = [dt.datetime(years[i],months[i],days[i]) for i in range(nVals)]; timeList = np.array(timeList)
  
    #interpolate to uniform time grid ("daily" values early on are every 2 days)
    tStart = timeList[0]; tEnd = timeList[-1]
    nDays = (tEnd-tStart).total_seconds()/(24*3600); nDays = int(nDays)
    print('{0} days between {1} {2}'.format(nDays, tStart, tEnd))
    times = [tStart+ dt.timedelta(days=i) for i in range(0, nDays)]; times = np.array(times)
  
    dTime = [(a-timeList[0]).total_seconds() for a in timeList]
    dTimeResample = [(a-timeList[0]).total_seconds() for a in times]
    extents = np.interp(dTimeResample, dTime, extents)    
    
    del years, months, days
    years = [i.year for i in times]
    months = [i.month for i in times]
    days = [i.day for i in times]
    
    years = np.array(years)
    months = np.array(months)
    days = np.array(days)

[inds] = np.where((years>=climo_years[0]) & (years<=climo_years[1]))

timefocus = times[inds]
yearfocus = years[inds]
monthfocus = months[inds]
dayfocus = days[inds]
extentfocus = extents[inds]

dextent = np.zeros_like(extentfocus).astype('f')
z0 = np.zeros_like(extentfocus).astype('f')
dextent[0] = float('NaN')
dextent[1:] = extentfocus[1:] - extentfocus[0:-1]
if plot_variable == 'dextent_int':
    ndays = 2
    dextent_int = np.zeros_like(extentfocus).astype('f')
    for ii in range(ndays,len(dextent_int)):
        dextent_int[ii] = np.nansum(dextent[ii-ndays+1:ii+1])
    dextent_int[0:ndays] = dextent_int[ndays]
elif plot_variable == 'dextent_diff':
    ndays = 2
    dextent_int = np.zeros_like(extentfocus).astype('f')
    dextent_int[2:] = extentfocus[2:] - extentfocus[0:-2]
    dextent_int[0:2] = dextent_int[2]
else:
    dextent_int = np.zeros_like(extentfocus).astype('f')
    dextent_int[:] = float('NaN')

if ( np.size(dayfocus) < 367) :
    dayinds = np.where(dayfocus==1)
    dayplot = (dayfocus[dayinds])
else:
    dayinds = np.where( (monthfocus==1) & (dayfocus==15) )
    dayplot = (yearfocus[dayinds])
    

datesave = []
for ii in range(0,len(dayplot)):
    if ( np.size(dayfocus) < 367) :
        datenow = str(ii+1) + '/ 01'
    else:
        datenow = str(int(dayplot[ii])) 
        
    #datenow = str(int(monthfocus[dayinds[ii]])) + '/' + str(int(dayfocus[dayinds[ii]]))
    datesave.append(datenow)



14852
16502 days between 1978-10-26 00:00:00 2023-12-31 00:00:00


In [5]:
yearsnow = [i.year for i in timefocus]
monthsnow = [i.month for i in timefocus]
daysnow = [i.day for i in timefocus]

daily_mean = []
daily_mean_d = []
daily_mean_dint = []
daily_mean_dint_ll = []
daily_mean_dint_ul = []
ul = errorbar_percentiles[0]
ll = errorbar_percentiles[1]
for mm in range(1,13):
    for dd in range(1,32):
        [indsnow] = np.where( (monthfocus==mm) & (dayfocus==dd))
        if np.size(indsnow)>0:
            extent_now = np.nanmean(extentfocus[indsnow])
            dextent_now = np.nanmean(dextent[indsnow])
            dextent_int_now = np.nanmean(dextent_int[indsnow])
            daily_mean.append(extent_now)
            daily_mean_d.append(dextent_now)
            daily_mean_dint.append(dextent_int_now)
            
            daily_mean_dint_ll_now = np.percentile(dextent_int[indsnow], ll)
            daily_mean_dint_ul_now = np.percentile(dextent_int[indsnow], ul)
            daily_mean_dint_ll.append(daily_mean_dint_ll_now)
            daily_mean_dint_ul.append(daily_mean_dint_ul_now)


In [6]:
if filter_climo == 'True':
    #butter_order = 3
    butter_order = 12
    sample_days = 1.0 # days
    lowcut_days = 18.0 # days
    
    #lowcut_days = 120.0
    highcut_days = 2.0 # days

    sample_rate = sample_days*24.0*3600.0 # convert to seconds
    lowcut = lowcut_days*24.0*3600.0 # convert to seconds
    highcut = highcut_days*24.0*3600.0 # convert to seconds

    sample_freq = 1.0/sample_rate
    lowcut_freq = 1.0/lowcut # convert to Hz
    highcut_freq = 1.0/highcut # convert to Hz
    nyq = 0.5*sample_freq

    lowcut = lowcut_freq/nyq
    highcut = highcut_freq/nyq

    b, a = butter(butter_order,lowcut,btype='highpass')
    #daily_mean = filtfilt(b,a,daily_mean)
    daily_mean_d = filtfilt(b,a,daily_mean_d)
    daily_mean_dint = filtfilt(b,a,daily_mean_dint)

In [8]:
outfile = open(climo_path,'w')
wcount = 0
if (compute_errorbars == 'False'):
    for ii in range(0,len(daily_mean)):
        print(ii)
        outfile.write('%d %7.4f %7.4f %7.4f\n' % (ii, daily_mean[ii], daily_mean_d[ii], daily_mean_dint[ii]))
else:
     for ii in range(0,len(daily_mean)):
        print(ii)
        outfile.write('%d %7.4f %7.4f %7.4f %7.4f %7.4f\n' % (ii, daily_mean[ii], daily_mean_d[ii], daily_mean_dint[ii], daily_mean_dint_ll[ii], daily_mean_dint_ul[ii]))   

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27